In [13]:
#!pip install selenium

In [1]:
import selenium
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

# Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
driver=webdriver.Chrome(r"chromedriver.exe")
url = "https://www.amazon.in/"
driver.get(url)

In [3]:
time.sleep(3)
search = driver.find_element_by_id("twotabsearchtextbox")     #fetching search bar and sending search item
item = str(input("Enter item name for search: "))
search.send_keys(item)
srch_btn = driver.find_element_by_id("nav-search-submit-button")
srch_btn.click()
time.sleep(5)

Enter item name for search: guitar


# Q2.In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [4]:
#fetching page urls
page_url = []
for i in driver.find_elements_by_xpath("//li[@class='a-selected' or @class='a-normal']//a"):
    page_url.append(i.get_attribute("href"))
time.sleep(5)
page_url
#required data to store
brand_name = []
name = []
ratings = []
num_ratings = []
prices = []
exchange = []
exp_delivery = []
availability = []
other_details = []
product_url = []
for x in page_url[0:3]: # 3 pages
    driver.get(x)
    time.sleep(3)
    urls = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-4']//a"):
        urls.append(i.get_attribute("href"))
    urls
    #fetch product urls 
    p_url = driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-4']//a")
    for i in p_url:
        product_url.append(i.get_attribute("href"))
        time.sleep(3)
        
    for i in urls:
        driver.get(i)
        time.sleep(3)
        # name of the product
        try:
            Name = driver.find_element_by_id("productTitle")
            name.append(Name.text)
        except NoSuchElementException:
            name.append('-')
        # Brand name
        try:
            brand = driver.find_element_by_xpath("//td[@class='a-span9']//span")
            brand_name.append(brand.text)
        except NoSuchElementException:
            brand_name.append('-')
        # ratings
        try:
            rating = driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']//span")
            ratings.append(rating.text)
        except NoSuchElementException:
            ratings.append('-')
        # number of ratings
        try:
            num_rating = driver.find_element_by_id("acrCustomerReviewText")
            num_ratings.append(num_rating.text)
        except NoSuchElementException:
            num_ratings.append('-')
        # price of the product
        try:
            price = driver.find_element_by_xpath("//td[@class='a-span12']//span")
            prices.append(price.text.replace('₹ ','Rs'))
        except NoSuchElementException:
            prices.append('-')
        # return or exchange
        try:
            exch = driver.find_element_by_id("RETURNS_POLICY")
            exchange.append(exch.text)
        except NoSuchElementException:
            exchange.append('-')
        # expected delivery data
        try:
            delivery = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']//b")
            exp_delivery.append(delivery.text)
        except NoSuchElementException:
            exp_delivery.append('-')
        # availability information
        try:
            avail = driver.find_element_by_xpath("//div[@id='availability']//span")
            availability.append(avail.text)
        except NoSuchElementException:
            availability.append('-')
        # other details
        try:
            oth_det = driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']//span")
            other_details.append(oth_det.text)
        except NoSuchElementException:
            other_details.append('-')
    


In [5]:
#store in a dataframe
data = list(zip(brand_name,name,ratings,num_ratings,prices,exchange,exp_delivery,availability,other_details,product_url))
df = pd.DataFrame(data, columns = ["BrandName","ProductName","Ratings","Number of Ratings","Price","Return or Exchange","ExpectedDelivery", "Availability", "OtherDetails","URL detail"])
df.to_csv(r"C:\Rajashri\Iternship\Task4 Selenioum\amazon_product.csv")
df

,BrandName,ProductName,Ratings,Number of Ratings,Price,Return or Exchange,ExpectedDelivery,Availability,OtherDetails,URL detail
0,Medellin,Medellin 38in Premium All Wood Acoustic Guitar...,3.9 out of 5,596 ratings,"₹3,199.00",7 Days Replacement,"Thursday, Sep 30",In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,"Kadence Guitar Frontier Series, Blue Acoustic ...",4 out of 5,285 ratings,"₹5,999.00",7 Days Replacement,"Thursday, Sep 30",In stock.,Product Type : Acoustic Guitar,https://www.amazon.in/gp/slredirect/picassoRed...
2,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5,"1,028 ratings","₹6,499.00",7 Days Replacement,"Thursday, Sep 30",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,ARCTIC,"ARCTIC Sky series 39"" Guitar (with Truss Rod) ...",3.6 out of 5,10 ratings,"₹5,364.00",7 Days Replacement,"Wednesday, Sep 29",In stock.,ARCTIC SKY series standard pack is an astoundi...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"13,077 ratings","₹2,649.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
175,YAMAHA,"Yamaha FS100C, 6-Strings Acoustic Guitar With ...",3.9 out of 5,8 ratings,"₹11,990.00",7 Days Replacement,"Saturday, Oct 2",Only 1 left in stock.,"Yamaha FS100C Acoustic Guitar with Sponge Bag,...",https://www.amazon.in/FS100C-6-Strings-Acousti...
176,-,"SUPERTIVE Kids Guitar, Beginner Toy Guitar for...",-,-,₹869.00,7 Days Replacement,"Thursday, Sep 30",In stock.,The perfect introduction to teaching kids abou...,https://www.amazon.in/gp/slredirect/picassoRed...
177,-,"Acoustic Guitars Decoration, Sturdy Durable Gu...",-,-,"₹2,239.00",10 Days Returnable,Oct 23 - 27,Only 2 left in stock.,"Great workmanship, the sound hole can match yo...",https://www.amazon.in/gp/slredirect/picassoRed...
178,-,"Guitar Pedal Board, Guitar Tuner Pedal LED for...",-,-,"₹34,639.00",7 Days Replacement,Nov 10 - 17,Usually dispatched in 2 to 3 weeks.,Supports limitless recording and overlapping o...,https://www.amazon.in/gp/slredirect/picassoRed...


# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [16]:
driver=webdriver.Chrome(r"chromedriver.exe")
url = "https://images.google.com/"
urls = []    # urls to download
data = []
search_item = ["fruits", "cars", "Machine Learning"]  #items list to search
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    search_bar = driver.find_element_by_tag_name("input") #finding search bar
    
    search_bar.send_keys(str(item))      #send search item
    
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click() #lets click on search button
    
    # scrolling the page in order to generate more images on the website
    for _ in range(200):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 300:
        break
    print("Downloading ImageNo{0}" .format(i))
    response= requests.get(urls[i])

    file = open(r"C:\Rajashri\Iternship\Task4 Selenioum\GoogleImages\ImgName"+str(i)+".jpg", "wb")

    file.write(response.content)

# Q4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [33]:
driver = webdriver.Chrome(r"chromedriver.exe")
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

cancle_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()
#giving inputs to search bar
serch_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
serch_bar.send_keys("smartphone")

srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

In [34]:
# urls of every smartphone
urls = []
for i in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    urls.append(i.get_attribute("href"))

# required data
brand_names = []
smartphone_name = []
color = []
RAM = []
storage = []
primary_camera = []
sec_camera = []
Display_size = []
display_resolution = []
processor = []
processor_core = []
battery_capacity = []
phone_price = []
product_url = []
#fetching Battery capacity
for i in driver.find_elements_by_xpath("//div[@class='fMghEO']"):
    try:
        battery_capacity1 = driver.find_element_by_xpath("//ul[@class='_1xgFaf']//li[4]")
        battery_capacity.append(battery_capacity1.text)
    except NoSuchElementException:
        battery_capacity.append('-')
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    # brand names
    try:
        br_name = driver.find_element_by_xpath("/html/body/div/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        brand_names.append(br_name.text.replace('Mobiles',''))
    except NoSuchElementException:
        brand_names.append('-')
    # smartphone names
    try:
        smart_name = driver.find_element_by_xpath("//h1[@class='yhB1nd']//span")
        smartphone_name.append(smart_name.text)
    except NoSuchElementException:
        smartphone_name.append('-')
    
    # colors of smartphone
    try:
        clr = driver.find_element_by_xpath("//table[@class='_14cfVK']//tr[4]//td[2]")
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
    time.sleep(2)
    #readmore button
    try:
        read_more_btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    except NoSuchElementException:
        pass
    time.sleep(3)
    
    # Display size
    try:
        disp_size = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[1]//td[2]")
        Display_size.append(disp_size.text)
    except NoSuchElementException:
        Display_size.append
        
    # display_resolution
    try:
        disp_res = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[2]//td[2]")
        display_resolution.append(disp_res.text)
    except NoSuchElementException:
        display_resolution.append('-')
        
    # processor info
    try:
        processor1 = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[2]//td[2]")
        processor.append(processor1.text)
    except NoSuchElementException:
        processor.append('-')
    # processor core 
    try:
        pro_core = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[3]//td[2]")
        processor_core.append(pro_core.text)
    except NoSuchElementException:
        processor_core.append('-')
    # storage/ROM
    try:
        rom = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[1]//td[2]")
        storage.append(rom.text)
    except NoSuchElementException:
        storage.append('-')
    # RAM info
    try:
        ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[2]//td[2]")
        RAM.append(ram.text)
    except NoSuchElementException:
        RAM.append('-')
    time.sleep(2)
    
    # primary camera information
    try:
        primary_camera1 = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[2]//td[2]")
        primary_camera.append(primary_camera1.text)
    except NoSuchElementException:
        primary_camera.append('-')
    # secondary camera info
    try:
        sec_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[5]//td[2]")
        sec_camera.append(sec_cam.text)
    except NoSuchElementException:
        sec_camera.append('-')
    
   
    # price 
    try:
        price1 = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        phone_price.append(price1.text)
    except NoSuchElementException:
        phone_price.append('-')
    
    


In [35]:
# dataframe
data = list(zip(brand_names,smartphone_name,color,RAM,storage,primary_camera,sec_camera,Display_size,display_resolution,processor,processor_core,battery_capacity,phone_price,urls))
df = pd.DataFrame(data, columns = ["Brand Name","Smartphone Name","Color","RAM","Storage(ROM)","primaryCamera","SecondaryCamera","Display size","Display Resolution","Processor","Processor core","Battery Capacity","Price","Product url"])
df.to_csv(r"C:\Rajashri\Iternship\Task4 Selenioum\phone.csv")
df

,Brand Name,Smartphone Name,Color,RAM,Storage(ROM),primaryCamera,SecondaryCamera,Display size,Display Resolution,Processor,Processor core,Battery Capacity,Price,Product url
0,POCO,"POCO C3 (Arctic Blue, 64 GB) (4 GB RAM)",Arctic Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh Li-ion Polymer Battery,"₹8,349",https://www.flipkart.com/poco-c3-arctic-blue-6...
1,realme,"realme C20 (Cool Blue, 32 GB) (2 GB RAM)",Cool Blue,MediaTek Helio G35,Android 10,2 GB,Dedicated Slot,"Handset, Adapter (5V/2A), USB Cable, SIM Card ...",RMX3063 / RMX3061,1600 x 720 Pixels,HD+,5000 mAh Li-ion Polymer Battery,"₹7,499",https://www.flipkart.com/realme-c20-cool-blue-...
2,realme,"realme C20 (Cool Grey, 32 GB) (2 GB RAM)",Cool Grey,MediaTek Helio G35,Android 10,2 GB,Dedicated Slot,"Handset, Adapter (5V/2A), USB Cable, SIM Card ...",RMX3063 / RMX3061,1600 x 720 Pixels,HD+,5000 mAh Li-ion Polymer Battery,"₹7,499",https://www.flipkart.com/realme-c20-cool-grey-...
3,POCO,"POCO M3 (Cool Blue, 64 GB) (4 GB RAM)",Cool Blue,4 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,5000 mAh Li-ion Polymer Battery,"₹11,149",https://www.flipkart.com/poco-m3-cool-blue-64-...
4,REDMI,"REDMI 9i (Nature Green, 64 GB) (4 GB RAM)",Nature Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh Li-ion Polymer Battery,"₹8,799",https://www.flipkart.com/redmi-9i-nature-green...
5,POCO,"POCO C3 (Lime Green, 32 GB) (3 GB RAM)",Lime Green,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh Li-ion Polymer Battery,"₹7,349",https://www.flipkart.com/poco-c3-lime-green-32...
6,POCO,"POCO C3 (Arctic Blue, 32 GB) (3 GB RAM)",Arctic Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh Li-ion Polymer Battery,"₹7,349",https://www.flipkart.com/poco-c3-arctic-blue-3...
7,OPPO,"OPPO A53 (Mint Cream, 64 GB) (4 GB RAM)",Mint Cream,4 GB,64 GB,13MP + 2MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh Li-ion Polymer Battery,"₹12,649",https://www.flipkart.com/oppo-a53-mint-cream-6...
8,OPPO,"OPPO A53 (Moonlight Black, 64 GB) (4 GB RAM)",Moonlight Black,4 GB,64 GB,13MP + 2MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh Li-ion Polymer Battery,"₹11,990",https://www.flipkart.com/oppo-a53-moonlight-bl...
9,realme,"realme C11 2021 (Cool Grey, 32 GB) (2 GB RAM)",Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,5000 mAh Li-ion Polymer Battery,"₹7,299",https://www.flipkart.com/realme-c11-2021-cool-...


# Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [38]:
import re
driver = webdriver.Chrome(r"chromedriver.exe")
url = 'https://www.google.co.in/maps/place/Pune,+Maharashtra/@18.5245646,73.7225394,11z/data=!3m1!4b1!4m5!3m4!1s0x3bc2bf2e67461101:0x828d43bf9d9ee343!8m2!3d18.5204303!4d73.8567437'
driver.get(url)

srarch_box = driver.find_element_by_class_name("tactile-searchbox-input")  # search bar

city_name = str(input("Enter City Name: "))  #Enter inputs to search for location

srarch_box.send_keys(city_name)    

submit = driver.find_element_by_class_name("searchbox-searchbutton").click()  #search button

currenturl = driver.current_url   #Current URL 

Latitude = re.findall('(\d+(\.\d*)?)', currenturl)[0]   # latitude
Longitude = re.findall('(\d+(\.\d*)?)', currenturl)[1]     # longitude

print("Geospatial coordinates of ", city_name + " are as follow: ")
print("Latitude :", Latitude[0])
print("Longitude :", Longitude[0])

Enter City Name: Delhi
Geospatial coordinates of  Delhi are as follow: 
Latitude : 18.5759985
Longitude : 73.9088407


# Q6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [24]:
driver = webdriver.Chrome(r"chromedriver.exe")
#get the web page with given url
url = "https://trak.in/india-startup-funding-investment-2015/"
driver.get(url)
time.sleep(3)



In [25]:
#lets unable table to show all entries
btn = driver.find_elements_by_xpath("//div[@class='dataTables_length']//select//option[4]")
for i in btn:
    i.click()
    time.sleep(4)

In [26]:
# required data from tables
dates = []
st_name = []
industry = []
sub_vertical = []
cities = []
investor = []
investment_type = []
amount = []
try:     #July 20 – September 20 dates only
    date = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[2]")
    for i in date:
        dates.append(i.text)
except NoSuchElementException:
    pass
# startup names
try:
    startup =  driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[3]")
    for i in startup:
        st_name.append(i.text)
except NoSuchElementException:
    pass
# industry names
try:
    ind = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[4]")
    for i in ind:
        industry.append(i.text)
except NoSuchElementException:
    pass
# sub verticals
try:
    sub_v = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[5]")
    for i in sub_v:
        sub_vertical.append(i.text)
except NoSuchElementException:
    pass
# city/location 
try:
    city = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[6]")
    for i in city:
        cities.append(i.text)
except NoSuchElementException:
    pass
# investor's name
try:
    inv_name = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[7]")
    for i in inv_name:
        investor.append(i.text)
except NoSuchElementException:
    pass
#  investment type
try:
    inv_type = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[8]")
    for i in inv_type:
        investment_type.append(i.text)
except NoSuchElementException:
    pass
# amount of investment
try:
    amt = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper' or @id='tablepress-49_wrapper' or @id ='tablepress-50_wrapper']//tr//td[9]")
    for i in amt:
        amount.append(i.text)
except NoSuchElementException:
    pass

#creaing dataframe
data = list(zip(dates,st_name,industry,sub_vertical,cities,investor,investment_type,amount))
df = pd.DataFrame(data, columns = ["Dates","StartupName","Industry/Vertical","SubVvertical","City","Investor's Name","Investment Type","Amount(in USD)"])
df.to_csv(r"C:\Rajashri\Iternship\Task4 Selenioum\fundingdata.csv")
df

,Dates,StartupName,Industry/Vertical,SubVvertical,City,Investor's Name,Investment Type,Amount(in USD)
0,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
1,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
2,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
3,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
4,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
5,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
6,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
7,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
8,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
9,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


# Q7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [8]:
driver = webdriver.Chrome(r"chromedriver.exe")
#get the web page with given url
url = "https://www.digit.in/"
driver.get(url)
time.sleep(3)
#lets search for best laptop
best_gaming_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

# info of laptops
laptop_name = []
oper_system = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_processor = []
prices = []
#   laptop names
name = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for i in name:
    laptop_name.append(i.text)
#   operating system
try:
    os = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for i in os:
        oper_system.append(i.text)
except NoSuchElementException:
    pass
try:
    disp = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for i in disp:
        display.append(i.text)
except NoSuchElementException:
    pass
try:
    pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for i in pro:
        processor.append(i.text)
except NoSuchElementException:
    pass
try:
    mem = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for i in mem:
        memory.append(i.text)
except NoSuchElementException:
    pass
try:
    wgt = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for i in wgt:
        weight.append(i.text)
except NoSuchElementException:
    pass
try:
    dim = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for i in dim:
        dimensions.append(i.text)
except NoSuchElementException:
    pass
try:
    g_pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for i in g_pro:
        graph_processor.append(i.text)
except NoSuchElementException:
    pass
try:
    price = driver.find_elements_by_xpath("//td[@class='smprice']")
    for i in price:
        prices.append(i.text.replace('₹','Rs '))
except NoSuchElementException:
    pass
data = list(zip(laptop_name,oper_system,display,processor,memory,weight,dimensions,graph_processor,prices))
df = pd.DataFrame(data, columns = ["Laptop Name","Operating System","Display","Processor","Memory","Weight(Kg)","Dimensions","Graphic-Processor","Price"])
df

,Laptop Name,Operating System,Display,Processor,Memory,Weight(Kg),Dimensions,Graphic-Processor,Price
0,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,N/A
1,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"Rs 341,990"
2,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,N/A
3,Asus ROG Zephyrus G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 164,990"
4,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
6,Acer Aspire 7 gaming,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 64,370"


# Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [27]:
driver = webdriver.Chrome(r"chromedriver.exe")

#get the web page with given url
url = "https://www.forbes.com/?sh=55783e032254"
driver.get(url)
time.sleep(3)


#lets get option button from the page
opt_btn = driver.find_element_by_xpath("//div[@class='header__left']//button").click()

#Now select Billionaires from options
blns = driver.find_elements_by_xpath("//li[@data-title='Billionaires']//a[1]")
bln = []
for i in blns[:1]:
    bln.append(i.get_attribute("href"))
     

for i in bln:
    driver.get(i)

bln_list = driver.find_element_by_xpath("//div[@class='channel-special-feature__articles-wrap']//a").click()
time.sleep(8)

#fetch required data from table
#fetching rank
rank = []
try:
    rnk = driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rnk:
        rank.append(i.text)
except NoSuchElementException:
    pass
#fetching names
names = []
try:
    name = driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        names.append(i.text)
except NoSuchElementException:
    pass
#fetching net worth
net_worth = []
try:
    worth = driver.find_elements_by_xpath("//div[@class='netWorth']")
    for i in worth:
        net_worth.append(i.text)
except NoSuchElementException:
    pass
#fetching age
Age = []
try:
    age = driver.find_elements_by_xpath("//div[@class='age']")
    for i in age:
        Age.append(i.text)
except NoSuchElementException:
    Age.append('-')
#fetching citizenship
citizenship = []
try:
    cit = driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in cit:
        citizenship.append(i.text)
except NoSuchElementException:
    citizenship.append('-')
#fetching source
source = []
try:
    src = driver.find_elements_by_xpath("//div[@class='source-column']")
    for i in src:
        source.append(i.text)
except NoSuchElementException:
    source.append('-')
#fetching Industry
industry = []
try:
    ind = driver.find_elements_by_xpath("//div[@class='category']")
    for i in ind:
        industry.append(i.text)
except NoSuchElementException:
    industry.append('-')
    
#creating dataframe
data = list(zip(rank,names,net_worth,Age,citizenship,source,industry))
df = pd.DataFrame(data, columns = ["Rank", "Name", "Net worth", "Age", "Citizenship", "Source","Industry"])
df

,Rank,Name,Net worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,,United States,Amazon,
1,2.,Elon Musk,$151 B,,United States,"Tesla, SpaceX",
2,3.,Bernard Arnault & family,$150 B,,France,LVMH,
3,4.,Bill Gates,$124 B,,United States,Microsoft,
4,5.,Mark Zuckerberg,$97 B,,United States,Facebook,
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,,Australia,real estate,
196,197.,Leonid Fedun & family,$11.1 B,,Russia,oil,
197,197.,Eyal Ofer,$11.1 B,,Israel,"real estate, shipping",
198,197.,Evan Spiegel,$11.1 B,,United States,Snapchat,


# Q9.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [7]:
driver = webdriver.Chrome(r"chromedriver.exe")

#get the web page with given url
url = "https://www.youtube.com/watch?v=e-ORhEE9VVg&t=3s"
driver.get(url)
time.sleep(3)

#Lets scroll the comments
for _ in range(10000):

    driver.execute_script("window.scrollBy(0,100)")

# required data
comments = []
upvotes = []
times = []
try:
    comment = driver.find_elements_by_xpath("//*[@id='content-text']")[:1000]
    for i in comment:
        comments.append(i.text)
except NoSuchElementException:
    comments.append('-')
time.sleep(3)

# upvotes
try:
    up = driver.find_elements_by_xpath("//*[@id='vote-count-middle']")[:1000]
    for i in up:
        upvotes.append(i.text)
except NoSuchElementException:
    comments.append('-')
    time.sleep(3)

# time when comment was posted
try:
    tim = driver.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text above-comment style-scope ytd-comment-renderer']")[:500]
    for i in tim:
        times.append(i.text.replace('(edited)',''))
except NoSuchElementException:
    times.append('-')
time.sleep(3)

# dataframe
data = list(zip(comments,upvotes,times))
df = pd.DataFrame(data, columns = ["Comment","Upvotes","Published time"])
df.head(500)

,Comment,Upvotes,Published time
0,Legend watch after 6 years,2.6K,1 week ago
1,"""I'm a nightmare dressed like a daydream""\nEve...",94K,1 year ago
2,her songs have that iconic touch that always m...,177,2 days ago
3,Legend watch after 6 years,200,1 day ago
4,Dear Future Generation Please Don't Let This T...,73,1 day ago
...,...,...,...
495,"""DARLING IM A NIGHTMARE DRESSED LIKE A DAYDREA...",430,8 months ago
496,I always got blank space for Taylor Swift,5,1 week ago
497,still a legend,,1 year ago
498,Am I the only one who searched “gotta love the...,1,2 days ago


# Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [26]:
driver = webdriver.Chrome(r"chromedriver.exe")
#get the web page with given url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(5)
search_job=driver.find_element_by_id('search-input-field')
search_job.send_keys("London")



In [28]:
enter = driver.find_element_by_xpath("//div[@class='label']").click()
time.sleep(5)
srch_btn = driver.find_element_by_xpath("//button[@id='search-button']").click()
time.sleep(10)

#lets find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []
for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(4)
    #fetching hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #fetching distance from city centre
    
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #fetch privates from price
        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #fetching dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
    #fetching facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #lets fetch url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #lets click on show more button for description
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    #fetching ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    #fetching total reviews
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
    #fetch overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #fetch property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

#creating dataframe
data = list(zip(hostel_name,distance,rating,reviews,over_all,pvt_prices,dorms_price,facilities,description))       
df = pd.DataFrame(data, columns = ["Hostel name","Distance from city centre","ratings","Total reviews","Overall review","Privates from price","Dorms from price","Facilities","Property Description"])
df

,Hostel name,Distance from city centre,ratings,Total reviews,Overall review,Privates from price,Dorms from price,Facilities,Property Description
0,The Finnish Church in London,4.5km from city centre,10,190,Superb,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","33 Albion Street, Rotherhithe, London, England"
1,Saint James Backpackers,5.5km from city centre,7.4,1763,Very Good,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","21 Longridge Road, London, England"
2,Wombat's The City Hostel London,3.6km from city centre,8.8,13189,Fabulous,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","7 Dock Street, London, England"
3,Smart Russell Square Hostel,2.6km from city centre,7.1,9510,Very Good,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","71 Guilford Street, London, England"
4,Urbany Hostel London,5.4km from city centre,9.8,123,Superb,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","48-49 Princes Square, Paddington, London, England"
5,Prime Backpackers Angel,3.6km from city centre,10,514,Superb,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","333 City Road, 333 City Road, London, England"
6,Smart Hyde Park View Hostel,5km from city centre,8.1,4481,Fabulous,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","16 Leinster Terrace W2 3EU, London, England"
7,Generator London,3km from city centre,7.6,6745,Very Good,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","Compton Place, Off 37 Tavistock Place, WC1, Lo..."
8,Smart Hyde Park Inn Hostel,5km from city centre,7.1,6245,Very Good,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","48 - 50 Inverness Terrace, Bayswater, London, ..."
9,Smart Camden Inn Hostel,4.4km from city centre,7.6,2687,Very Good,Rs4547,Rs2637,"Free WiFi, Follows Covid-19 sanitation guidance","55/57 Bayham Street, Camden, London, England"
